# Mapping the probability distribution of Hydrogen atom in 2D

In [ ]:
import os
import math
import sympy
import numpy as np
import scipy.special as ss

import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
odir = './out/'
ffmt = 'pdf'
fdpi = 200

colors = [
    'tab:red',
    'tab:orange',
    'tab:green',
    'tab:blue',
    'tab:purple'
]

# Set axtick dimensions
major_size = 5
minor_size = 3
major_width = minor_width = 1
for tick in ['xtick', 'ytick']:
    mpl.rcParams[f'{tick}.major.size'] = major_size
    mpl.rcParams[f'{tick}.major.width'] = major_width
    mpl.rcParams[f'{tick}.minor.size'] = minor_size
    mpl.rcParams[f'{tick}.minor.width'] = minor_width
mpl.rcParams['text.usetex'] = False

## Physical constans

In [ ]:
# 1. Reduced Planck's constant [J s]
hbar = 1.054571817e-34
# 2. Vacuum permittivity [F] = [C^2 J^-1]
eps_0 = 8.8541878128e-12
# 3. Rest mass of electron [kg]
m_e = 9.1093837015e-31
# 4. Rest mass of proton [kg]
m_p = 1.67262192369e-27
# 5. Reduced mass of e-p system
mu = 1/(1/m_e + 1/m_p)
# 6. Elementary charge [C]
q_e = 1.602176634e-19

# 7. (Derived from others) Reduced Bohr radius [m]
a_0 = 4 * np.pi * eps_0 * hbar**2 / (mu * q_e**2)

## Generalized Laguerre polynomial

For arbitrary real $\alpha$ the polynomial solutions of the differential equation

$$
x y'' + \left( 1 + \alpha - x \right) y' + ny = 0
$$
are called generalized Laguerre polynomials. One can also define the generalized Laguerre polynomials recursively, defining the first two polynomials as

$$
L_{0}^{\alpha} \left( x \right) = 1
$$
$$
L_{1}^{\alpha} \left( x \right) = 1 + \alpha - x
$$
and then using the following recurrence relation for any $k \geq 1$:

$$
L_{k+1}^{\alpha} \left( x \right)
=
\frac{\left( 2k + 1 + \alpha - x \right) L_{k}^{\alpha} \left( x \right) - \left( k + \alpha \right) L_{k-1}^{\alpha} \left( x \right)}{k + 1}
$$

In [ ]:
def laguerre(X, alpha, n):
    '''
    Calculates the generalized Laguerre polynomial of `k`th order.
    '''
    if n == 0:
        return np.ones_like(X)
    elif n == 1:
        return 1 + alpha - X
    else:
        # Here we calculate the n-th Laguerre polynomial
        # The 'n' input parameter thus becomes 'k+1' in the equation above
        # This means, that:
        #   - k+1 := n
        #   - k   := n-1
        #   - k-1 := n-2
        return (
            (2 * (n-1) + 1 + alpha - X) * laguerre(X, alpha, (n-1)) \
            - \
            ((n-1) + alpha) * laguerre(X, alpha, (n-2))
        )/n

In [ ]:
def plot_laguerre(X, k_max=4):

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))
    ax.grid(True, lw=1, ls='--', alpha=0.6)

    axisticksize = 15
    axislabelsize = 20
    axislegendsize = 15

    for k in range(0, k_max+1):
        L = laguerre(X, alpha=0.0, n=k)
        ax.plot(X, L, label=f'$L_{k}^{{(0)}}$', lw=3, c=colors[k])
    for k in range(0, k_max+1):
        L = laguerre(X, alpha=1.0, n=k)
        ax.plot(X, L, label=f'$L_{k}^{{(1)}}$', lw=3, ls='--', c=colors[k])

    ax.set_xlim(-2.5,10)
    ax.set_ylim(-5,10)

    ax.set_xlabel('$X$', fontsize=axislabelsize)
    ax.set_ylabel('$L_{k}^{(\\alpha)} (X)$', fontsize=axislabelsize)
    ax.tick_params(axis='both', which='major', labelsize=axisticksize)

    ax.legend(loc='upper center', fontsize=axislegendsize, ncol=2)

    os.makedirs(odir, exist_ok=True)
    plt.savefig(os.path.join(odir, f'Laguerre_kmax-{k_max}.{ffmt}'),
                dpi=fdpi, bbox_inches='tight')

    plt.show()

In [ ]:
X_laguerre = np.linspace(-10, 10, 100)

In [ ]:
%%time
plot_laguerre(X_laguerre)

## Associated Legendre polynomials

The canonical solutions of the general Legendre equation

$$
\frac{d}{dx}
\left[
    \left( 1 - x^{2} \right) \frac{d}{dx} P_{l}^{m} \left( x \right)
\right]
+
\left[
    l \left( l + 1 \right) - \frac{m^{2}}{1 - x^{2}} P_{l}^{m} \left( x \right)
\right]
=
0
$$

are called the associated Legendre polynomials. One can give an arbitrary Legendre polynomial $P_{l}^{m} \left( x \right)$ with non-negative integer $l$ and $m$, where $0 \leq m \leq l$ in terms of derivatives of ordinary Legendre polynomials:

$$
P_{l}^{m} \left( x \right)
=
\left( -1 \right)^{m} \left( 1 - x^{2} \right)^{m/2} \frac{\operatorname{d}^{m}}{\operatorname{d}x^{m}} \left[ P_{l} \left( x \right) \right]
$$

Since the Rodrigues' formula states, that the ordinary Legendre polynomial is

$$
P_{l} \left( x \right)
=
\frac{1}{2^{l} l!} \frac{\operatorname{d}^{l}}{\operatorname{d}x^{l}} \left[ \left( x^{2} - 1 \right)^{l} \right]
$$

the equation for the associated Legendre polynomials could be reshaped as

$$
P_{l}^{m} \left( x \right)
=
\frac{\left( -1 \right)^{m}}{2^{l} l!}
\left( 1 - x^{2} \right)^{m/2} \frac{\operatorname{d}^{l+m}}{\operatorname{d}x^{l+m}} \left[ \left( x^{2} - 1 \right)^{l} \right]
$$

The definition of associated Legendre polynomials could be extended to negative values of $m$:

$$
P_{l}^{-m} \left( x \right)
=
\left( -1 \right)^{m} \frac{\left( l - m \right)!}{\left( l + m \right)!} P_{l}^{m} \left( x \right)
$$
Thus $m$ could take the integer values, where $-l \leq m \leq l$.

In [ ]:
def legendre(X, l, m):
    '''TODO:
    '''
    # In case of l == m == 0, the Legendre polynomial is a constant indetity
    if l == m == 0:
        return np.ones_like(X)

    # In every other case, calculate the non-constant Legendre-polynomial
    negative = (m < 0)  # Final equation depends on the sign of `m`
    m = np.abs(m)       # Final equation derived for non-negative `m` values

    # Symbolically differentiate (x^2 - 1)^l, (m + l) times
    x_s, l_s, m_s = sympy.symbols('x l m')
    diff_s = sympy.diff((x_s**2 - 1)**l_s, x_s, l + m)
    diff_s = diff_s.subs({'l': l, 'm': m})
    # Calculate the Legendre function for m >= 0 and optionally m < 0
    P_c = (-1)**m_s / (2**l_s * sympy.factorial(l_s)) * (1 - x_s**2)**(m_s/2)
    if negative:
        P_c *= 1/(-1)**(m_s) * sympy.factorial(l_s - m_s) / sympy.factorial(l_s + m_s)
    P_c = P_c.subs({'l': l, 'm': m})
    P_s = sympy.simplify(P_c * diff_s)
    P_s = P_s.subs({'l': l, 'm': m})
    # Convert the expression to a numpy-compatible lambda function
    P = sympy.lambdify([x_s], P_s, 'numpy')

    # Evaluate the Legendre polynomial for all `x` values
    P = np.array(P(X))

    return P

In [ ]:
def plot_legendre(X, m):

    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 8))
    ax.grid(True, lw=1, ls='--', alpha=0.6)

    axisticksize = 15
    axislabelsize = 20
    axislegendsize = 15

    for l in range(m, m+5):
        ax.plot(X, legendre(X, l=l, m=m),
                lw=3, c=colors[l-m], label=f'$P_{l}^{{(m)}} (X)$')

    ax.set_xlabel('$X$', fontsize=axislabelsize)
    ax.set_ylabel('$P_{l}^{m} (X)$', fontsize=axislabelsize)
    ax.tick_params(axis='both', which='major', labelsize=axisticksize)

    ax.legend(loc='lower right', fontsize=axislegendsize)

    os.makedirs(odir, exist_ok=True)
    plt.savefig(os.path.join(odir, f'Legendre_lmax_{l}-m_{m}.{ffmt}'),
                dpi=fdpi, bbox_inches='tight')
    plt.show()

In [ ]:
X_legendre = np.linspace(-1, 1, 101)

In [ ]:
%%time
plot_legendre(X=X_legendre, m=0)

In [ ]:
%%time
plot_legendre(X=X_legendre, m=1)

In [ ]:
%%time
plot_legendre(X=X_legendre, m=2)

## Spherical harmonics

In quantum mechanics we usually define the spherical harmonics as

$$
Y_{l}^{m} \left( \vartheta, \varphi \right)
=
\left( -1 \right)^{m}
\sqrt{\frac{\left( 2l + 1 \right)}{4 \pi} \frac{\left( l - m \right)!}{\left( l + m \right)!}}
P_{l}^{m} \left( \cos \left( \vartheta \right) \right) e^{i m \varphi}
$$

In [ ]:
def Ylm(theta, phi, *, l : int, m : int):
    '''
    Calculates Laplace's spherical harmonics. Uses Python's sympy for symbolic
    calculations to evaluate the Legendre polynomials.

    Parameters
    ----------
    theta : float or array-like
        Azimuthal angles to evaluate the spherical harmonics over.
    phi : float or array-like
        Polar angles to evalate the spherical harmonics over.
    l : int
        Azimuthal quantum number. Could take the value of any integer larger
        or equals to 0 and smaller than `n`.
    m : int
        Magnetic quantum number. Could take the value of any integer between
        `-l <= m <= l`.
    '''
    # Condon–Shortley phase
    CSP = (-1)**m

    # First part : The normalization factor (term with the square root)
    Y_1 = np.sqrt(((2*l + 1) * math.factorial(l - m))/(4 * np.pi * math.factorial(l + m)))

    # Second part : The Legendre polynomial and exponential
    Y_2 = legendre(np.cos(theta), l, m) * np.exp(1j * m * phi)

    return CSP * Y_1 * Y_2

In [ ]:
def plot_Ylm(m, l_n, *, phi=0, n_points=1000):
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))

    axisticksize = 15
    axislabelsize = 20
    axislegendsize = 15
    scattersize = 3

    theta = np.linspace(0, 2*np.pi, n_points)
    for l in range(m, m+l_n+1):
        ax.scatter(theta, np.abs(Ylm(theta, phi, l=l, m=m)),
                   c=colors[l-m], s=scattersize**2, label=f'$Y_{{{l}, {m}}} (\\vartheta, \\varphi=0)$')

    ax.set_xlabel('$\\vartheta$', fontsize=axislabelsize)
    ax.set_ylabel('$Y_{lm} (\\vartheta, \\varphi=0)$', fontsize=axislabelsize, labelpad=30)
    ax.tick_params(axis='both', which='major', labelsize=axisticksize)

    ax.legend(loc='upper right', fontsize=axislegendsize)

    os.makedirs(odir, exist_ok=True)
    plt.savefig(os.path.join(odir, f'Y_lmax_{l}-m_{m}.{ffmt}'),
                dpi=fdpi, bbox_inches='tight')
    plt.show()

In [ ]:
%%time
plot_Ylm(m=0, l_n=4, phi=0, n_points=1000)

In [ ]:
%%time
plot_Ylm(m=1, l_n=4, phi=0, n_points=1000)

In [ ]:
%%time
plot_Ylm(m=2, l_n=4, phi=0, n_points=1000)

## Wave function of H

By solving the radial Schrödinger equation the wave function of an arbitrary atom could be expressed as:

$$
\psi_{nlm} \left( r, \vartheta, \varphi \right)
=
R_{n,l} \left( r \right)
\cdot
Y_{l,m} \left( \vartheta, \varphi \right)
$$

Where in the case of the hydrogen atom, the $R_{n,l}$ radial part is the following
$$
R_{n,l} \left( r \right)
=
\sqrt{\left( \frac{2}{n a_{0}} \right)^{3} \frac{\left( n - l - 1 \right)!}{2n \left[ \left( n + 1 \right)! \right]}}
e^{-\rho / 2} \rho^{l} L_{n - l - 1}^{2 l + 1} \left( \rho \right)
$$

Here $\rho = 2r\ /\ \left( n a_{0} \right)$, where $a_{0} = 4 \pi \varepsilon_{0} \hbar^{2}\ /\ \left( \mu \left| q_{e} \right| \left| q_{p} \right| \right)$ is the reduced Bohr radius and $\mu$ is the reduced mass of the electron-proton system.

References:

[1] : Csanád Máté. *Atomfizika*. 2017. p.65. http://atomfizika.elte.hu/atomkvantum/files/atomfiz_vazlat.pdf

In [ ]:
def init_grid(N : int, L : float):
    '''
    Creates a grid to evaluate the wave function on.

    Parameters
    ----------
    N : int
        Number of cells on the grid.
    L : float
        The side length of the simulation grid in meters.
        Typical value for atoms should be around around 1e-10.
    '''
    X = np.linspace(-L/2, L/2, N)
    Z = np.linspace(L/2, -L/2, N)

    return np.array(np.meshgrid(X, Z))

In [ ]:
def Rnl(r, *, n : int, l : int):
    '''
    Calculates the radial term in the hydrogene atom's wave function.

    Parameters
    ----------
    r : float or array-like
        Radial distance of simulation cells from the center of the simulation
    n : int
        Principal quantum number. Could take the value of any integer larger
        than 0.
    l : int
        Azimuthal quantum number. Could take the value of any integer larger
        than 0 and smaller than `n`.
    '''
    # First part : The normalization factor (term with the square root)
    rnl_1 = np.sqrt((2/(n*a_0))**3 * math.factorial(n - l - 1)/(2*n*math.factorial(n + l)))
    # Second part : The exponential and rho^{l}
    rho = 2*r/(n*a_0)
    rnl_2 = np.exp(-rho/2) * rho**l
    # Third part : Laguerre polynomial
    rnl_3 = laguerre(rho, alpha=2*l+1, n=n-l-1)
    return rnl_1 * rnl_2 * rnl_3

In [ ]:
def Psi(XZ, n, l, m, *,
        phi=0, return_A=False, return_log=False):
    '''TODO:
    '''
    # Variables to calculate: r, theta; phi is set
    # 1. r: 2D distances from center
    r = np.sqrt(XZ[0]**2 + XZ[1]**2)
    # 2D angles along x-z plane:
    # :math:`theta = arctan(dz/dx)`
    # Calculate all `theta` angles for each pixels of the grid
    theta = np.arctan2(XZ[0], XZ[1]) + np.pi

    psi = Rnl(r, n=n, l=l) * Ylm(theta, phi, l=l, m=m)

    if return_log:
        psi = np.nan_to_num(np.log10(psi))
    if return_A:
        psi = np.abs(psi/np.max(psi))**2
    return psi

In [ ]:
def plot_H(n, l, m, *,
           N=400, L=1e-10, plot_A=True, plot_log=False,
           axis=False, grid=False, cbar=False, text=False, save=False):
    '''TODO:
    '''
    # Initialize grid
    XZ = init_grid(N=N, L=L)
    # Calculate the wavefunction
    psi = Psi(XZ, n, l, m, return_A=plot_A, return_log=plot_log)

    fig, ax = plt.subplots(figsize=(11, 11), facecolor='black',
                           subplot_kw=dict(facecolor='black'))
    if not axis:
        ax.axis('off')
    if grid:
        ax.grid(True, ls='--', color='white', alpha=0.6)

    axiscbarfontsize = 16
    axisticksize = 13
    axislabelsize = 18

    im = ax.imshow(psi.real, cmap='magma', vmin=0, vmax=1,
                   extent=[-L, L, L, -L], interpolation='none')

    if text:
        ax.text(x=0.8, y=0.95, s=f'n={n}, l={l}, m={m}', c='white',
                fontsize=28, fontweight='demibold',
                horizontalalignment='center', verticalalignment='center',
                transform=ax.transAxes,
                bbox=dict(facecolor='black', alpha=0.2, lw=0))

    ax.set_xlabel('X coordinate [m]', fontsize=axislabelsize, color='white')
    ax.set_ylabel('Z coordinate [m]', fontsize=axislabelsize, color='white')

    ax.tick_params(axis='both', which='major', labelsize=axisticksize, colors='white')
    ax.tick_params(axis='x', which='major', rotation=42)

    # create an axes on the right side of ax. The width of cax will be 5%
    # of ax and the padding between cax and ax will be fixed at 0.05 inch.
    if cbar:
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='5%', pad=0.1)
        cbar = plt.colorbar(mappable=im, cax=cax)
        cbar.ax.tick_params(labelsize=axiscbarfontsize, colors='white')

    if save:
        os.makedirs(odir, exist_ok=True)
        pad_inches = 0 if not axis else 0.1
        plt.savefig(os.path.join(odir, f'orbit_n_{n}-l_{l}-m_{m}.png'),
                    format='png', dpi=200,
                    facecolor='black', edgecolor='black',
                    pad_inches=pad_inches, bbox_inches='tight')
    plt.show()

In [ ]:
plot_H(n=4, l=3, m=0,
       N=400, L=3.5e-09, plot_A=True, plot_log=False,
       axis=False, grid=False, cbar=False, text=False, save=True)